In [ ]:
import redcap
import pandas as pd
from temba_client.v2 import TembaClient
from temba_client.exceptions import TembaRateExceededError, TembaTokenError, TembaHttpError, TembaNoSuchObjectError
import math
import numpy as np
import json
from support_functions import * 

Define classes to contain the different methods that will be used

In [ ]:
redcap = RedCap()
rapidpro = rapidPro()

In [ ]:
missing = []
form_names = []
for form in form_names:
    missing_temp = []
    redcap.read_redcap_credentials_file(credential_file = 'redcapcredentials.json',form = form)
    rapidpro.read_rapidpro_credentials_file(credential_file = 'rapidprocredentials.json')
    data_records = redcap.get_records()
    data_records_flat = redcap.get_records_flat()
    data_metadata = redcap.get_metadata()
    expected = redcap.get_expected_fields()
    completed = redcap.get_completed_fields()
    missing_temp = redcap.get_missing_fields()
    missing.append(missing_temp)


Get the record data and metadata

In [ ]:
missing_fields = pd.DataFrame.from_dict(missing).T
missing_fields.columns = form_names

Next we need to parse the phone number from the Redcap database and check it works -- it needs to be converted into the correct format for the message sender

In [ ]:
import phonenumbers
import types

for index,row in data_records_flat.iterrows():
    wa_number = row['whatsapp_number']
    print(wa_number)
    try:
        f_wa_number = phonenumbers.parse(wa_number, None)
        phone_number = ('tel:'+phonenumbers.format_number(f_wa_number,
                                                          phonenumbers.PhoneNumberFormat.E164))
        print(phone_number)
        contacts = rapidpro.client.get_contacts(urn = phone_number)
        #if the contact exists then make sure it is in the correct group
        if contacts.first() is None:
            print('contact doesnt exist- creating')
            rapidpro.client.create_contact(urns = [phone_number])
            rapidpro.add_contact_to_groups(phone_number,'asos-investigators')
        else:
            print('contact already exists- just updating')
            rapidpro.add_contact_to_groups(phone_number,'asos-investigators')
        number_missing_fields = len(missing[index])
        print(number_missing_fields)
        name = row['name']
        if(number_missing_fields > 0):
            string = 'Thanks {name} for helping us by registering for the ASOS-2 trial. We have noticed that there are {missing_vals} missing fields in your latest survey. Would you mind completing the registration when you are able? Thanks!'.format(missing_vals=number_missing_fields,name=name)
        else:
            string = 'Thanks {name} for helping us by registering for the ASOS-2 trial. You have completed the latest survey. Thanks!'.format(form_name = redcap.form,name=name)

        print(string)
        message_dict = {'wa_message' : string }
        rapidpro.add_fields_to_contact(phone_number,message_dict)
        rapidpro.start_flow(phone_number)
    except:
        print('failed')
        print(wa_number)

In [ ]:
payload_records = {
        'token': redcap.redcap_token,
        'content': 'record',
        'format': 'json',
        'type': 'flat',
        'rawOrLabel': 'Label',
        'rawOrLabelHeaders': 'Label',
        'exportCheckboxLabel': 'false',
        'exportSurveyFields': 'true',
        'exportDataAccessGroups': 'false',
        'returnFormat': 'json',
        'forms': redcap.form
    }

data_records = post(redcap.redcap_url, data=payload_records)

In [ ]:
records_df = pd.DataFrame.from_dict(data_records.json())

In [ ]:
records_df

Here we update all the contacts as necessary